In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

Vertex AI管道：评估AutoML Tabular分类模型的批量预测结果

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/model_evaluation/automl_tabular_classification_model_evaluation.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/model_evaluation/automl_tabular_classification_model_evaluation.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/model_evaluation/automl_tabular_classification_model_evaluation.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI Workbench中打开
    </a>
  </td>                                                                                               
</table>

## 概述

本笔记本演示如何使用Vertex AI分类模型评估组件来评估AutoML Tabular分类模型。模型评估有助于根据评估指标确定您的模型的性能，并在必要时改进模型。

了解有关[Vertex AI模型评估](https://cloud.google.com/vertex-ai/docs/evaluation/introduction)的更多信息。了解有关[表格数据分类](https://cloud.google.com/vertex-ai/docs/tabular-data/classification-regression/overview)的更多信息。

### 目标

在本教程中，您将学习如何训练一个 Vertex AI AutoML Tabular 分类模型，并通过使用 `google_cloud_pipeline_components` 在 Vertex AI pipeline 作业中评估它：

本教程使用以下 Google Cloud ML 服务和资源：

- Vertex AI `Datasets`
- Vertex AI `Training`（AutoML Tabular 分类）
- Vertex AI `Model Registry`
- Vertex AI `Pipelines`
- Vertex AI `Batch Predictions`

执行的步骤包括：

- 创建一个 Vertex AI `Dataset`。
- 在 `Dataset` 资源上训练一个 AutoML Tabular 分类模型。
- 将训练好的 `AutoML 模型资源` 导入到 pipeline 中。
- 运行一个 `Batch Prediction` 作业。
- 使用 `Classification Evaluation component` 评估 AutoML 模型。
- 将分类指标导入到 AutoML 模型资源中。

### 数据集

此笔记本中使用的数据集是PetFinder数据集的一部分，可在Kaggle上的[此处](https://www.kaggle.com/c/petfinder-adoption-prediction)找到。当前数据集只是原始数据集的一部分，用于预测宠物是否被领养。它包括以下字段：

- `Type`：动物类型（1 = 狗，2 = 猫）
- `Age`：列出时宠物的年龄，以月为单位
- `Breed1`：宠物的主要品种
- `Gender`：宠物的性别
- `Color1`：宠物的颜色1
- `Color2`：宠物的颜色2
- `MaturitySize`：成熟时的大小（1 = 小型，2 = 中型，3 = 大型，4 = 特大型，0 = 未指定）
- `FurLength`：毛长（1 = 短毛，2 = 中毛，3 = 长毛，0 = 未指定）
- `Vaccinated`：宠物是否已接种疫苗（1 = 是，2 = 否，3 = 不确定）
- `Sterilized`：宠物是否绝育（1 = 是，2 = 否，3 = 不确定）
- `Health`：健康状况（1 = 健康，2 = 轻伤，3 = 重伤，0 = 未指定）
- `Fee`：领养费（0 = 免费）
- `PhotoAmt`：为该宠物上传的照片总数
- `Adopted`：宠物是否被领养（是/否）。

**注意**：此数据集已从公共云存储桶转移，可以在此笔记本中访问。

### 成本
本教程使用了谷歌云的可计费组件：

* Vertex AI
* 云存储

了解[Vertex AI价格](https://cloud.google.com/vertex-ai/pricing)和[云存储价格](https://cloud.google.com/storage/pricing)，并使用[Pricing Calculator](https://cloud.google.com/products/calculator/)根据您的预计使用情况生成成本估算。

安装以下所需的软件包以执行此笔记本。

In [ ]:
! pip3 install --upgrade --quiet google-cloud-aiplatform \
                                 google-cloud-pipeline-components==1.0.26 \
                                 matplotlib

### 仅限Colab：取消注释以下单元格以重新启动内核

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

## 开始之前

### 设置您的项目ID

**如果您不知道您的项目ID**，请尝试以下方法：
* 运行 `gcloud config list`。
* 运行 `gcloud projects list`。
* 参考支持页面：[查找项目ID](https://support.google.com/googleapi/answer/7014113)。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

#### 区域

您也可以更改 Vertex AI 使用的 `REGION` 变量。了解更多关于 [Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations) 的信息。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

### 验证您的 Google Cloud 账户

根据您的 Jupyter 环境，您可能需要手动进行身份验证。请按照以下相关说明操作。

**1. Vertex AI Workbench**
* 不需要进行任何操作，因为您已经通过身份验证。

**2. 本地 JupyterLab 实例，请取消注释并运行：**

In [ ]:
# ! gcloud auth login

3. 协作，取消注释并运行:

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

查看如何在https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples 上为您的服务帐户授予云存储权限。

将整个笔记本中使用的云服务启用。

运行下面的单元格以启用计算引擎、容器注册表和Vertex AI服务。

In [ ]:
!gcloud services enable compute.googleapis.com         \
                       containerregistry.googleapis.com  \
                       aiplatform.googleapis.com

创建一个云存储桶

创建一个存储桶，用于存储中间产物，如数据集。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

只有当您的存储桶不存在时，请运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_URI

#### 服务账号

您可以使用服务账号来创建 Vertex AI 流水线作业。如果您不想使用项目的计算引擎服务账号，请将 `SERVICE_ACCOUNT` 设置为另一个服务账号 ID。

In [ ]:
SERVICE_ACCOUNT = "[your-service-account]"  # @param {type:"string"}

In [ ]:
import sys

IS_COLAB = "google.colab" in sys.modules

if (
    SERVICE_ACCOUNT == ""
    or SERVICE_ACCOUNT is None
    or SERVICE_ACCOUNT == "[your-service-account]"
):
    # Get your service account from gcloud
    if not IS_COLAB:
        shell_output = !gcloud auth list 2>/dev/null
        SERVICE_ACCOUNT = shell_output[2].replace("*", "").strip()

    else:  # IS_COLAB:
        shell_output = ! gcloud projects describe  $PROJECT_ID
        project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
        SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"

    print("Service Account:", SERVICE_ACCOUNT)

为Vertex AI Pipelines设置服务账户访问权限

运行以下命令，为您的服务账户授予读取和写入管道工件的权限，这些工件位于您在上一步中创建的存储桶中。您只需要针对每个服务账户运行此步骤一次。

In [ ]:
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectCreator $BUCKET_URI

! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectViewer $BUCKET_URI

### 导入库

导入Vertex AI Python SDK和其他所需的Python库。

In [ ]:
import json

import google.cloud.aiplatform as aiplatform
import matplotlib.pyplot as plt
from google.cloud import aiplatform_v1

### 初始化用于Python的Vertex AI SDK

为您的项目和相应的存储桶初始化用于Python的Vertex AI SDK。

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)

创建一个在 Vertex AI 中使用数据集来源创建的托管表格数据集资源。

In [ ]:
DATA_SOURCE = "gs://cloud-samples-data/ai-platform-unified/datasets/tabular/petfinder-tabular-classification.csv"

In [ ]:
# Create the Vertex AI Dataset resource
dataset = aiplatform.TabularDataset.create(
    display_name="petfinder-tabular-dataset",
    gcs_source=DATA_SOURCE,
)

print("Resource name:", dataset.resource_name)

用创建的数据集训练一个简单的分类模型，目标列为 `Adopted`。

使用Vertex AI SDK的 `AutoMLTabularTrainingJob` 类创建一个训练作业。为训练作业设置一个显示名称，并指定列转换的适当数据类型。

In [ ]:
TRAINING_JOB_DISPLAY_NAME = "[your-train-job-display-name]"  # @param {type:"string"}

In [ ]:
# If no display name is specified, use the default one
if (
    TRAINING_JOB_DISPLAY_NAME == ""
    or TRAINING_JOB_DISPLAY_NAME is None
    or TRAINING_JOB_DISPLAY_NAME == "[your-train-job-display-name]"
):
    TRAINING_JOB_DISPLAY_NAME = "train-petfinder-automl"

### 创建一个AutoML表格训练任务

`AutoMLTabularTrainingJob`类使用以下参数创建一个AutoML训练任务：

- `display_name`：Vertex AI训练任务的可读名称。
- `optimization_prediction_type`：Vertex AI模型要生成的预测类型。例如：回归，分类。
- `column_specs`（可选）：应用于输入列的转换（包括数据类型更正）。
- `optimization_objective`：要最小化或最大化的优化目标。根据预测类型而定，选择相应的参数。如果未设置该字段，则使用默认的优化目标函数。

了解更多关于[AutoMLTabularTrainingJob类](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.AutoMLTabularTrainingJob)。

In [ ]:
# Define the AutoML training job
train_job = aiplatform.AutoMLTabularTrainingJob(
    display_name=TRAINING_JOB_DISPLAY_NAME,
    optimization_prediction_type="classification",
    column_specs={
        "Type": "categorical",
        "Age": "numeric",
        "Breed1": "categorical",
        "Color1": "categorical",
        "Color2": "categorical",
        "MaturitySize": "categorical",
        "FurLength": "categorical",
        "Vaccinated": "categorical",
        "Sterilized": "categorical",
        "Health": "categorical",
        "Fee": "numeric",
        "PhotoAmt": "numeric",
    },
)

设置一个显示名称为自动ML表格分类模型。

In [ ]:
MODEL_DISPLAY_NAME = "[your-model-display-name]"  # @param {type:"string"}

In [ ]:
# If no name is specified, use the default name
if (
    MODEL_DISPLAY_NAME == ""
    or MODEL_DISPLAY_NAME is None
    or MODEL_DISPLAY_NAME == "[your-model-display-name]"
):
    MODEL_DISPLAY_NAME = "pet-adoption-prediction-model"

### 运行训练任务

现在，通过传递以下参数来在创建的表格数据集上运行训练任务：

- `dataset`：需要用来训练 Vertex AI 模型的同一项目中的 Vertex AI 表格数据集。
- `target_column`：Vertex AI 模型需要预测的列的名称。
- `model_display_name`：生成的 Vertex AI 模型的显示名称。
- `budget_milli_node_hours`（可选）：创建 Vertex AI 模型的训练预算，以毫节点小时表示，即该字段中的值为1,000表示1个节点小时。模型的训练成本不会超过此预算。

了解更多关于[`AutoMLTabularTrainingJob类的run()方法`](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.AutoMLTabularTrainingJob#google_cloud_aiplatform_AutoMLTabularTrainingJob_run)。

训练任务大约需要1.5 - 2小时才能完成。

In [ ]:
# Specify the target column
target_column = "Adopted"

# Run the training job
model = train_job.run(
    dataset=dataset,
    target_column=target_column,
    model_display_name=MODEL_DISPLAY_NAME,
    budget_milli_node_hours=1000,
)

##列出训练时的模型评估

在训练工作完成后，获取模型评估并打印出来。

In [ ]:
# Get evaluations
model_evaluations = model.list_model_evaluations()

# Print the evaluation metrics
for evaluation in model_evaluations:
    evaluation = evaluation.to_dict()
    print("Model's evaluation metrics from training:\n")
    metrics = evaluation["metrics"]
    for metric in metrics.keys():
        print(f"metric: {metric}, value: {metrics[metric]}\n")

## 运行用于模型评估的管道

现在，您可以运行一个 Vertex AI 批量预测作业，并使用管道生成其结果的评估和特征归因。

为此，您可以通过调用 `evaluate` 函数来创建一个 Vertex AI 管道。了解更多关于[evaluate函数](https://github.com/googleapis/python-aiplatform/blob/main/google/cloud/aiplatform/models.py#L5127)的信息。

### 定义运行评估函数的参数

指定运行 `evaluate` 函数所需的参数。

以下是 `evaluate` 函数参数的说明：

- `prediction_type`：这次评估运行所解决的问题类型。'classification' 和 'regression' 是目前支持的问题类型。
- `target_field_name`：用作分类目标的列的名称。
- `gcs_source_uris`：批量预测的输入实例的Cloud Storage存储桶URI列表。
- `class_labels`：目标列中的类标签列表。
- `generate_feature_attributions`：可选。模型评估作业是否应生成特征归因。如果未指定，则默认为 False。

In [ ]:
job = model.evaluate(
    prediction_type="classification",
    target_field_name=target_column,
    gcs_source_uris=[DATA_SOURCE],
    class_labels=["No", "Yes"],
    generate_feature_attributions=True,
)
print("Waiting model evaluation is in process")
job.wait()

## 模型评估管道的结果

在之前步骤的结果中，点击生成的链接以在云控制台中查看您的运行情况。

在用户界面中，当您点击某些管道有向无环图（DAG）节点时，许多节点会展开或折叠。以下是DAG的部分展开视图（点击图像查看更大版本）。

<img src="images/automl_tabular_classification_evaluation_pipeline.PNG">

### 获取模型评估结果

在评估管道完成后，运行以下单元格以打印评估指标。

In [ ]:
model_evaluation = job.get_model_evaluation()

In [ ]:
# Iterate over the pipeline tasks
for (
    task
) in model_evaluation._backing_pipeline_job._gca_resource.job_detail.task_details:
    # Obtain the artifacts from the evaluation task
    if (
        ("model-evaluation" in task.task_name)
        and ("model-evaluation-import" not in task.task_name)
        and (
            task.state == aiplatform_v1.types.PipelineTaskDetail.State.SUCCEEDED
            or task.state == aiplatform_v1.types.PipelineTaskDetail.State.SKIPPED
        )
    ):
        evaluation_metrics = task.outputs.get("evaluation_metrics").artifacts[0]
        evaluation_metrics_gcs_uri = evaluation_metrics.uri

print(evaluation_metrics)
print(evaluation_metrics_gcs_uri)

### 可视化指标

使用条形图可视化可用的指标，如`auRoc`和`logLoss`。

In [ ]:
metrics = []
values = []
for i in evaluation_metrics.metadata.items():
    metrics.append(i[0])
    values.append(i[1])
plt.figure(figsize=(5, 3))
plt.bar(x=metrics, height=values)
plt.title("Evaluation Metrics")
plt.ylabel("Value")
plt.show()

获取特征归因

运行下面的单元格以打印特征归因。

In [ ]:
# Iterate over the pipeline tasks
for (
    task
) in model_evaluation._backing_pipeline_job._gca_resource.job_detail.task_details:
    # Obtain the artifacts from the feature attribution task
    if (task.task_name == "feature-attribution") and (
        task.state == aiplatform_v1.types.PipelineTaskDetail.State.SUCCEEDED
        or task.state == aiplatform_v1.types.PipelineTaskDetail.State.SKIPPED
    ):
        feat_attrs = task.outputs.get("feature_attributions").artifacts[0]
        feat_attrs_gcs_uri = feat_attrs.uri

print(feat_attrs)
print(feat_attrs_gcs_uri)

通过获取特征属性的云存储URI，获取属性值。

In [ ]:
# Load the results
attributions = !gsutil cat $feat_attrs_gcs_uri

# Print the results obtained
attributions = json.loads(attributions[0])
print(attributions)

### 可视化特征归因

使用条形图可视化每个特征的获得的归因。

In [ ]:
data = attributions["explanation"]["attributions"][0]["featureAttributions"]
features = []
attr_values = []
for key, value in data.items():
    features.append(key)
    attr_values.append(value)

plt.figure(figsize=(5, 3))
plt.bar(x=features, height=attr_values)
plt.title("Feature Attributions")
plt.xticks(rotation=90)
plt.ylabel("Attribution value")
plt.show()

清理工作

要清理此项目中使用的所有Google Cloud资源，您可以[删除用于教程的Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在此教程中创建的各个资源。

将 `delete_bucket` 设置为 **True** 以创建在此笔记本中创建的Cloud Storage存储桶。

In [ ]:
import os

# Delete model resource
model.delete()

# Delete the dataset resource
dataset.delete()

# Delete the training job
train_job.delete()

# Delete the evaluation pipeline
job.delete()

# Delete Cloud Storage objects
delete_bucket = False
if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil -m rm -r $BUCKET_URI